## Phillip Paiva Brito - 20160143171
## Jorge Luiz Pereira da Silva Filho - 20180090431
## Thomas Vitor P.  Monteiro - 20200014649

In [2]:
def adicionar_nova_sequencia_ao_dicionario(dicionario, nova_sequencia):
    proximo_codigo = len(dicionario)
    dicionario[nova_sequencia] = proximo_codigo

In [3]:
def criar_dicionario_inicial():
    dicionario = {}
    for i in range(256):
        dicionario[(i.to_bytes(1,byteorder='big',signed=False))] = i
    return dicionario

In [4]:
def writeInstance(filePath,saida):
    file = open(filePath,"wb")
    
    for i in saida:
        file.write(i.to_bytes(2,byteorder='big'))

    file.close()

In [5]:
def writeInstanceD(filePath,saida,tamanhomaximo):
    file = open(filePath,"wb")
    
    for i in saida:
        file.write(i)

    file.close()

In [6]:
def compressao_LZW(nome_arquivo, output, tamanho_maximo):
    dicionario = criar_dicionario_inicial()
    tam_max_dic = 2 ** tamanho_maximo
    buffer = b''
    codigos = []

    with open(nome_arquivo, 'rb') as arquivo:
        while True:
            byte = arquivo.read(1)

            if not byte:
                break
            
            nova_sequencia = buffer + byte

            if nova_sequencia in dicionario:
                buffer = nova_sequencia
            else:
                if(len(dicionario) < tam_max_dic): # verifica tamanho maximo do dicionario
                    adicionar_nova_sequencia_ao_dicionario(dicionario, nova_sequencia)
                codigos.append(dicionario[buffer])
                buffer = byte

    if buffer:
        codigos.append(dicionario[buffer])
    
    writeInstance(output, codigos)
    return

In [7]:
def descompressao_LZW(filepath, output , tamanho_maximo):
    dicionario = criar_dicionario_inicial()
    tam_max_dic = 2 ** tamanho_maximo
    codigos = []
    buffer = b''
    
    with open(filepath, 'rb') as arquivo:
        while True:
            byte = arquivo.read(2)
            if not byte:
                break

            aux = int.from_bytes(byte,'big')
            try:
                codigo = list(dicionario.keys())[list(dicionario.values()).index(aux)] # retorna a chave correspondente ao indice
            except: # indice não encontrado no dicionario
                codigo = buffer + buffer[:1]

            nova_sequencia = buffer + codigo

            if nova_sequencia in dicionario:
                buffer = nova_sequencia
            else:
                nova_sequencia = buffer + codigo[:1] # nova sequencia concatena as sequncias anteriores com a primeira letra do codigo
                if(len(dicionario) < tam_max_dic):
                    adicionar_nova_sequencia_ao_dicionario(dicionario, nova_sequencia) # adiciona sequencia ao dicionario
            buffer = codigo
            codigos.append(codigo)
    writeInstanceD(output, codigos, 1)
    
    
    return

In [8]:
compressao_LZW("teste.txt","testecomp.bin",16)

In [9]:
descompressao_LZW("testecomp.bin", "testeDesc.txt",16)

In [10]:
compressao_LZW("corpus16MB.txt", "compCorpus16MB_16.bin", 16)

In [11]:
descompressao_LZW("compCorpus16MB_16.bin", "descompCorpus16MB_16.txt",16)